# Sreamlit 사용 웹 챗봇과 AI 에이전트 구현

- Streamlit은 파이썬만으로 대화형 웹 앱을 쉽게 만들 수 있는 오픈소스 프레임워크이다. <br>
버튼, 슬라이더, 입력창 같은 UI를 간단한 코드로 추가할 수 있고 데이터 시각화도 바로 지원한다. <br>
streamlit run app.py 명령으로 로컬 웹 서버를 실행해 결과를 브라우저에서 확인할 수 있다.<br>

- https://streamlit.io/  <br>
https://velog.io/@euisuk-chung/Streamlit-%EC%86%8C%EA%B0%9C-%EB%B0%8F-%ED%99%9C%EC%9A%A9-%EA%B0%80%EC%9D%B4%EB%93%9C

In [ ]:
# streamlit 모듈 설치
# ! pip install streamlit

# 코드 실행 방법 : Jupyter Notebook에서는 실행 안되고 터미널 창(Anaconda Prompt)에서만 실행 가능
# 실행 명령 예) streamlit run streamlit_test.py

In [1]:
# from dotenv import load_dotenv
# import os

# # .env 파일의 내용 불러오기
# load_dotenv("C:/env/.env")

# # 환경 변수 가져오기
# API_KEY = os.getenv("OPENAI_API_KEY")

# from openai import OpenAI
# client = OpenAI(api_key=API_KEY)

In [5]:
%%writefile streamlit_test.py

from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

import streamlit as st

# ======================
# 1. OpenAI 클라이언트 초기화
# ======================
from openai import OpenAI
client = OpenAI(api_key=API_KEY)

# ======================
# 2. Streamlit 페이지 설정
# ======================
st.set_page_config(page_title="OpenAI Chatbot", page_icon="🤖", layout="centered")
st.title("🤖 OpenAI Chatbot with Streamlit")
st.markdown("간단한 **대화형 챗봇** 예제입니다. 모델 선택, temperature 조절, 대화 기록 저장 기능을 포함합니다.")

# ======================
# 3. 사이드바 옵션
# ======================
st.sidebar.header("⚙️ 설정")
model = st.sidebar.selectbox(
    "모델 선택",
    ["gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"],
    index=0
)
temperature = st.sidebar.slider("창의성 (temperature)", 0.0, 1.5, 0.7, 0.1)
max_tokens = st.sidebar.slider("최대 토큰 수", 50, 1000, 300, 50)

# ======================
# 4. 세션 상태 초기화 (대화 기록)
# ======================
if "messages" not in st.session_state:
    st.session_state.messages = []

# ======================
# 5. 기존 대화 기록 출력
# ======================
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# ======================
# 6. 사용자 입력
# ======================
if user_input := st.chat_input("질문을 입력하세요..."):
    # 사용자 메시지 저장
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # OpenAI API 호출
    with st.chat_message("assistant"):
        with st.spinner("🤔 답변 생성 중..."):
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": m["role"], "content": m["content"]} for m in st.session_state.messages],
                temperature=temperature,
                max_tokens=max_tokens
            )
            answer = response.choices[0].message.content
            st.markdown(answer)

    # 어시스턴트 응답 저장
    st.session_state.messages.append({"role": "assistant", "content": answer})

    # 토큰 사용량 표시
    if hasattr(response, "usage"):
        st.sidebar.markdown("---")
        st.sidebar.markdown(f"**🔢 사용된 토큰**")
        st.sidebar.markdown(f"- prompt_tokens: {response.usage.prompt_tokens}")
        st.sidebar.markdown(f"- completion_tokens: {response.usage.completion_tokens}")
        st.sidebar.markdown(f"- total_tokens: {response.usage.total_tokens}")

# 윈도우 탐색기에서 C:/ 경로에 streamlit_test.py 파일을 복사
# Anaconda Prompt 실행
# cd c:/
# streamlit run streamlit_test.py

Overwriting streamlit_test.py


### Streamlit 으로 Langchain 에이전트 구현

In [12]:
%%writefile streamlit_langchain_test.py
import os
from dotenv import load_dotenv
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool


# -------------------------------------------------------
# 0. 환경설정
# -------------------------------------------------------
load_dotenv("C:/env/.env")

st.set_page_config(page_title="LangChain AI Agent Web App", page_icon="🤖")
st.title("🤖 LangChain AI Agent Web App")
st.markdown("LangChain v1.0 + Streamlit 예제")

user_input = st.text_input("질문을 입력하세요:", "")

# -------------------------------------------------------
# 1. Tool 정의
# -------------------------------------------------------
@tool
def multiply(a: int, b: int) -> int:
    """두 수를 곱한 값을 반환한다."""
    return a * b

@tool
def get_weather(city: str) -> str:
    """도시 이름을 받아 예시용 날씨를 반환한다."""
    return f"{city}의 날씨는 맑음이다."

# -------------------------------------------------------
# 2. LLM 및 에이전트 구성
# -------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_agent(
    model=llm,
    tools=[multiply, get_weather],
    system_prompt=(
        "너는 사용자의 요청을 해결하기 위해 필요시 제공된 도구를 반드시 호출해야 하는 에이전트이다. "
        "도구를 직접 호출하지 않고 임의로 결과를 만들어내지 않는다."
    ),
)

# -------------------------------------------------------
# 3. 실행 및 출력
# -------------------------------------------------------
if st.button("에이전트 실행") and user_input.strip():
    with st.spinner("에이전트가 응답 중입니다..."):
        result = agent.invoke({
            "messages": [
                {"role": "user", "content": user_input}
            ]
        })
        st.subheader("결과:")
        st.write(result["messages"][-1].content)

# 사용 예시:
#  "부산의 날씨를 알려줘"
#  "2와 5를 곱한 결과는?"
#  "서울의 날씨와 3×7 결과를 알려줘"

# 윈도우 탐색기에서 C:/ 경로에 streamlit_langchain_test.py 파일을 복사  
# Anaconda Prompt 실행
# cd c:/
# streamlit run streamlit_langchain_test.py

Overwriting streamlit_langchain_test.py


## Streamlit 사용피드백 루프형 워크플로우 에이전트 구현

In [13]:
%%writefile agents_feedback_graph.py

# agents_feedback_graph.py
# ==============================================================
# LangGraph 기반 피드백 루프형 워크플로우
# ==============================================================
import os
from dotenv import load_dotenv
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

# 환경 설정
load_dotenv("C:/env/.env")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# --------------------------------------------------------------
# 상태 정의
# --------------------------------------------------------------
class AgentState(TypedDict):
    idea: str
    analysis: str
    content: str
    review: str
    feedback: str
    revised: str

# --------------------------------------------------------------
# Analyzer Node
# --------------------------------------------------------------
def analyzer_node(state: AgentState) -> AgentState:
    prompt = f"""
    다음 아이디어를 한 문장으로 요약하고, 핵심 키워드 3개를 추출하라.
    아이디어: {state['idea']}
    """
    result = llm.invoke(prompt)
    state["analysis"] = result.content.strip()
    return state

# --------------------------------------------------------------
# Writer Node (1차 작성)
# --------------------------------------------------------------
def writer_node(state: AgentState) -> AgentState:
    prompt = f"""
    아래 분석 내용을 바탕으로 3문장 이내의 홍보 문구를 작성하라:
    {state['analysis']}
    """
    result = llm.invoke(prompt)
    state["content"] = result.content.strip()
    return state

# --------------------------------------------------------------
# Reviewer Node (1차 평가)
# --------------------------------------------------------------
def reviewer_node(state: AgentState) -> AgentState:
    prompt = f"""
    다음 문구를 평가하라.
    - 명확성 (0~10)
    - 창의성 (0~10)
    - 문법 정확도 (0~10)
    - 개선 제안을 1문장으로 제시하라.
    문구:
    {state['content']}
    """
    result = llm.invoke(prompt)
    review_text = result.content.strip()
    state["review"] = review_text
    # 개선 제안만 추출
    state["feedback"] = review_text.split("\n")[-1]
    return state

# --------------------------------------------------------------
# Writer Node (피드백 반영 재작성)
# --------------------------------------------------------------
def rewriter_node(state: AgentState) -> AgentState:
    prompt = f"""
    다음 피드백을 반영하여 문구를 개선하라.
    기존 문구:
    {state['content']}

    피드백:
    {state['feedback']}
    """
    result = llm.invoke(prompt)
    state["revised"] = result.content.strip()
    return state

# --------------------------------------------------------------
# Reviewer Node (최종 평가)
# --------------------------------------------------------------
def final_reviewer_node(state: AgentState) -> AgentState:
    prompt = f"""
    아래 수정된 문구를 다시 평가하라.
    - 명확성 / 창의성 / 문법 정확도 점수를 다시 제시하라.
    문구:
    {state['revised']}
    """
    result = llm.invoke(prompt)
    state["review"] += "\n\n[최종 평가]\n" + result.content.strip()
    return state

# --------------------------------------------------------------
# LangGraph 구성 (피드백 루프 포함)
# --------------------------------------------------------------
def build_feedback_workflow():
    graph = StateGraph(AgentState)

    graph.add_node("analyzer", RunnableLambda(analyzer_node))
    graph.add_node("writer", RunnableLambda(writer_node))
    graph.add_node("reviewer", RunnableLambda(reviewer_node))
    graph.add_node("rewriter", RunnableLambda(rewriter_node))
    graph.add_node("final_reviewer", RunnableLambda(final_reviewer_node))

    # 흐름 연결
    graph.add_edge("analyzer", "writer")
    graph.add_edge("writer", "reviewer")
    graph.add_edge("reviewer", "rewriter")         # 리뷰 후 피드백 반영
    graph.add_edge("rewriter", "final_reviewer")   # 개선 후 최종 평가
    graph.set_entry_point("analyzer")
    graph.set_finish_point("final_reviewer")

    return graph.compile()

feedback_workflow = build_feedback_workflow()


Writing agents_feedback_graph.py


In [20]:
%%writefile app_streamlit.py

# app_streamlit.py
import streamlit as st
from agents_feedback_graph import feedback_workflow

# 페이지 설정
st.set_page_config(page_title="LangGraph 피드백 루프 에이전트", layout="centered")

# ------------------------------------------------------------
# ✨ 커스텀 CSS (폰트 크기, 중앙 정렬, 줄간격 개선)
# ------------------------------------------------------------
st.markdown("""
<style>
    /* 제목 전체 정렬 */
    .main-title {
        text-align: center;
        font-size: 2rem;             /* 기존보다 작게 */
        font-weight: 700;
        margin-bottom: 0.2em;
        line-height: 1.2;
    }

    /* 부제목(워크플로우 단계 표시) */
    .subtitle {
        text-align: center;
        font-size: 0.9rem;
        color: #6c757d;
        margin-bottom: 1.5em;
    }

    /* 입력 안내 문구 */
    label[data-testid="stTextAreaLabel"] {
        font-weight: 600;
        font-size: 1rem;
    }

    /* 버튼 가운데 정렬 */
    div.stButton > button {
        display: block;
        margin: 0 auto;
        width: 180px;
    }

    /* 결과 영역 개선 */
    .stSuccess {
        text-align: center;
    }
</style>
""", unsafe_allow_html=True)

# ------------------------------------------------------------
# 제목 + 부제목
# ------------------------------------------------------------
st.markdown('<h1 class="main-title">🔄 LangGraph 피드백 루프 기반 AI 에이전트</h1>', unsafe_allow_html=True)
st.markdown('<div class="subtitle">Analyzer → Writer → Reviewer → Rewriter → Final Reviewer 순서로 자동 실행</div>', unsafe_allow_html=True)

# ------------------------------------------------------------
# 입력 영역
# ------------------------------------------------------------
user_input = st.text_area("💡 아이디어를 입력하세요:", height=100)

if st.button("워크플로우 실행"):
    if not user_input.strip():
        st.warning("아이디어를 입력해주세요.")
    else:
        with st.spinner("에이전트가 협업 중입니다..."):
            state = {"idea": user_input}

            for event in feedback_workflow.stream(state):
                if isinstance(event, tuple) and len(event) == 2:
                    step, output = event
                else:
                    step, output = "unknown", event

                if step == "analyzer":
                    st.markdown("### 🔍 Step 1: Analyzer 결과")
                    st.write(output.get("analysis", "결과 없음"))
                elif step == "writer":
                    st.markdown("### ✍️ Step 2: Writer (1차 문구)")
                    st.write(output.get("content", "결과 없음"))
                elif step == "reviewer":
                    st.markdown("### 🧾 Step 3: Reviewer (1차 평가)")
                    st.write(output.get("review", "결과 없음"))
                elif step == "rewriter":
                    st.markdown("### ♻️ Step 4: Rewriter (피드백 반영)")
                    st.write(output.get("revised", "결과 없음"))
                elif step == "final_reviewer":
                    st.markdown("### ✅ Step 5: Final Reviewer (최종 평가)")
                    st.write(output.get("review", "결과 없음"))

            # 최종 결과 출력
            st.divider()
            st.markdown("## 🏁 최종 결과 요약")
            result = feedback_workflow.invoke(state)
            st.write(result.get("revised", "최종 수정된 문구 없음"))
            st.write(result.get("review", "최종 평가 없음"))

            st.success("🎉 워크플로우 완료!")


Overwriting app_streamlit.py


In [ ]:
# 윈도우 탐색기에서 위에서 생성된 2개의 파일을 C:/ 경로에복사  
# Anaconda Prompt 실행
# cd c:/
# streamlit run app_streamlit.py

# 질문 예제:
# 친환경 소재로 만든 가벼운 백팩을 홍보하는 문구를 만들어줘
# 퇴근길 사람들에게 희망을 주는 짧은 광고 문구를 만들어줘
# 플라스틱 줄이기 환경 캠페인을 위한 SNS 홍보 문구를 만들어줘
# 혈당을 실시간으로 분석해주는 스마트워치 기능을 홍보하는 짧은 문구를 만들어줘

In [27]:
# 이 소스는 Jupyter Notebook에서 실행

# ==============================================================
# LangGraph 워크플로우 시각화 JSON Export (완전 호환 버전)
# ==============================================================

import json
from agents_feedback_graph import build_feedback_workflow

# LangGraph 인스턴스 생성
workflow = build_feedback_workflow()

# ✅ 내부 그래프 객체 가져오기
graph_obj = workflow.get_graph()  # Graph 객체

graph_json = {
    "nodes": [],
    "edges": []
}

# ✅ 노드 리스트 수집
for node_name in graph_obj.nodes.keys():
    graph_json["nodes"].append({
        "id": node_name,
        "type": "node",
        "label": node_name
    })

# ✅ 엣지 리스트 수집 (networkx 호환형 접근)
try:
    # 최신 버전에서는 edges.data() 로 엣지 순회
    for source, target, data in graph_obj.edges.data():
        graph_json["edges"].append({
            "source": source,
            "target": target
        })
except Exception:
    # 구버전 대비 fallback
    for edge in graph_obj.edges:
        if isinstance(edge, (tuple, list)):
            source = edge[0]
            target = edge[1] if len(edge) > 1 else None
            if target:
                graph_json["edges"].append({
                    "source": source,
                    "target": target
                })

# ✅ JSON 파일 저장
with open("langgraph_workflow.json", "w", encoding="utf-8") as f:
    json.dump(graph_json, f, indent=4, ensure_ascii=False)

print("✅ LangGraph 구조가 langgraph_workflow.json 파일로 저장되었습니다.")


✅ LangGraph 구조가 langgraph_workflow.json 파일로 저장되었습니다.


In [28]:
# Jupyter Notebook에서는 실행
# LangGraph 출력
from IPython.display import Markdown, display

with open("langgraph_workflow.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Mermaid 그래프 문자열 생성
mermaid = "```mermaid\ngraph LR\n"
for edge in data["edges"]:
    mermaid += f"    {edge['source']} --> {edge['target']}\n"
mermaid += "```"

display(Markdown(mermaid))  

```mermaid
graph LR
    __start__ --> analyzer
    analyzer --> writer
    reviewer --> rewriter
    rewriter --> final_reviewer
    writer --> reviewer
    final_reviewer --> __end__
```